In [1]:
import cvxpy as cp
import numpy as np

In [2]:
def find_covariance_matrix_m(S_target_np, P_matrices_np, M_dim, solver_name='SCS'):

    M = cp.Variable((M_dim, M_dim), symmetric=True)
    sum_term = 0
    for P_i in P_matrices_np:
        sum_term += P_i.T @ M @ P_i

    objective = cp.Minimize(cp.norm(sum_term - S_target_np, 'fro'))
    
    constraints = [M >> 0]
    problem = cp.Problem(objective, constraints)
    try:
        problem.solve(solver=solver_name)
    except Exception as e:
        print(f"An unexpected error occurred during solving: {e}")
        return None, None, "Error"
    if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        return M.value, problem.value, problem.status
    else:
        print(f"Problem did not solve to optimality. Status: {problem.status}")
        print("This could mean the problem is infeasible, unbounded, or the solver failed to converge.")
        return None, problem.value, problem.status


In [9]:
#np.random.seed(42) # for reproducibility

M_dim = 7 # Increased dimension of the square matrix M

# 1. Define a 'true' M (which is PSD) to simulate the scenario
# Create a random matrix A and then M = A @ A.T to ensure M is PSD
random_matrix_for_M = np.random.randn(M_dim, M_dim)
true_M_np = random_matrix_for_M @ random_matrix_for_M.T

print(f"True M (for demonstration, dimension {M_dim}x{M_dim}):\n{np.round(true_M_np, 4)}")


num_permutations = 4 # Number of permutation matrices
list_of_P_matrices_np = []
for _ in range(num_permutations):
    # np.random.permutation creates a random permutation of indices
    # np.eye(M_dim)[indices] creates a permutation matrix
    permutation_indices = np.random.permutation(M_dim)
    P_i = np.eye(M_dim)[permutation_indices]
    list_of_P_matrices_np.append(P_i)

print(f"\nUsing {len(list_of_P_matrices_np)} permutation matrices (randomly generated).")
print(list_of_P_matrices_np)


S_target_generated_np = np.zeros((M_dim, M_dim))
for P_i in list_of_P_matrices_np:
    S_target_generated_np += P_i.T @ true_M_np @ P_i

#print(f"\nGenerated S (Target for solver):\n{S_target_generated_np}")

found_M_np, optimal_value, status = find_covariance_matrix_m(
        S_target_generated_np, list_of_P_matrices_np, M_dim
    )


if found_M_np is not None:
     print(f"\n--- Solver Results ---")
     print(f"Found M by solver:\n{found_M_np}")
     print(f"Optimal objective value (Frobenius norm of difference): {optimal_value:.4e}")
     print(f"Solver status: {status}")

     # Verify the solution properties
     '''
     reconstructed_S_np = np.zeros((M_dim, M_dim))
     for P_i in list_of_P_matrices_np:
        reconstructed_S_np += P_i.T @ found_M_np @ P_i
     print(f"\nReconstructed S using found M:\n{reconstructed_S_np}")'''

     # Check if the found M is close to the true M (if S was perfectly generated and M is unique)
     print(f"Is found M close to true M (np.allclose)? {np.allclose(found_M_np, true_M_np, atol=1e-6)}")
     print(f"Max abs difference between found M and true M: {np.max(np.abs(found_M_np - true_M_np)):.4e}")

     # Verify if found M is symmetric and PSD
     is_symmetric = np.allclose(found_M_np, found_M_np.T)
     print(f"Is found M symmetric? {is_symmetric}")
     found_eigenvalues = np.linalg.eigvals(found_M_np)
     is_psd = np.all(found_eigenvalues >= -1e-6) # Allow small negative for numerical precision
     print(f"Is found M PSD? {is_psd}. Eigenvalues: {found_eigenvalues}")
else:
     print("\nCould not find M.")



True M (for demonstration, dimension 7x7):
[[ 7.6799 -2.6332 -0.1386 -0.0464 -1.6664 -3.4359 -2.1391]
 [-2.6332  3.9799  2.3753 -2.3949 -0.0274 -0.8451  1.6778]
 [-0.1386  2.3753  6.9284 -0.1823  4.2148 -2.5472  1.8476]
 [-0.0464 -2.3949 -0.1823  7.8226  3.5323  1.3506  0.3013]
 [-1.6664 -0.0274  4.2148  3.5323 10.465   0.0309  6.6307]
 [-3.4359 -0.8451 -2.5472  1.3506  0.0309  3.3049 -0.038 ]
 [-2.1391  1.6778  1.8476  0.3013  6.6307 -0.038   9.0239]]

Using 4 permutation matrices (randomly generated).
[array([[0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.]]), array([[0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       